$${\color{yellow}{\text{Applied Linear Algebra: Variance maximization using PyTorch}}}$$



---

Load essential libraries

---

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline
import sys
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

In [ ]:
X=np.array([76,126,38],[74,120,38])

---

Mount Google Drive folder if running Google Colab

---

In [ ]:
## Mount Google drive folder if running in Colab
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    DIR = '/content/drive/MyDrive/Colab Notebooks/ALA'
    DATA_DIR = DIR+'/Data/'
else:
    DATA_DIR = 'Data/'

Mounted at /content/drive


---

Load the food texture dataset

---

In [ ]:
## Load the food texture dataset
FILE = DATA_DIR + 'food-texture.csv'
df_food = pd.read_csv(FILE, index_col = 0, header = 0)
df_food.head(5)

,Oil,Density,Crispy,Fracture,Hardness
B110,16.5,2955,10,23,97
B136,17.7,2660,14,9,139
B171,16.2,2870,12,17,143
B192,16.7,2920,10,31,95
B225,16.3,2975,11,26,143


---

Preprocess the dataset

---

In [ ]:
## Create a list of continuous and categorical column names
continuous_cols = ['Oil', 'Density', 'Fracture', 'Hardness']
categorical_cols = ['Crispy']

# Typecasting columns to correct types
df_food[categorical_cols] = df_food[categorical_cols].astype('category')
df_food[continuous_cols] = df_food[continuous_cols].astype('float64')

## Print dataframe column types
df_food.dtypes

,0
Oil,float64
Density,float64
Crispy,category
Fracture,float64
Hardness,float64


---

Using PyTorch, calculate an optimal direction $\mathbf{v}$ (a vector with unit magnitude) such that the variance of the projected values $$\dfrac{1}{n}\sum_{i=1}^n\left(\underbrace{\mathbf{x}^{(i)}\cdot\mathbf{v}}_{\text{projection of }i\text{th sample}}-\underbrace{\pmb{\mu}\cdot\mathbf{v}}_{\text{average of projected samples = projection of average sample}}\right)^2$$ is maximized.

The direction vector that you will get as the answer from this cell should match with the answer from the next cell where we do PCA using the in-built sklearn library.

---

In [ ]:
# Data matrix (select only continuous columns)
X = torch.tensor(df_food[["Oil", "Density", "Fracture", "Hardness"]].values, dtype = torch.float64)

# Mean sample
mu = torch.mean(X, axis=0)
X_centered = X - mu

# Initial direction vector (has to be a unit vector)
w = torch.tensor(np.ones(X.shape[1]), dtype = torch.float64, requires_grad = True)
with torch.no_grad():
  w.data = w.data / torch.norm(w)


# Define optimizer (try different optimizers if answers don't match)
optimizer = torch.optim.Adam(, lr = ?)

# Loss function
def loss_fn(w):
  loss = -torch.mean(torch.square(? - ?))
  return loss

# Optimization loop
num_epochs = 100
for epoch in range(num_epochs):
  # Zero out the gradients
  optimizer.zero_grad()

  # Loss calculation
  loss = ?

  # Backward propagation and optimization
  loss.?
  ?.step()

  # Print the loss every 2 epochs
  if epoch%2 == 0:
    print(f'Epoch {epoch}, loss = {loss.item()}')

  # Constraint satisfaction (w should be unit vector)
  with torch.no_grad():
    w.data = ? / ?

# Print the optimized direction vector
print(w)

---

PCA using sklearn module (just run the cell to get the optimized direction vector)

---

In [ ]:
from sklearn.decomposition import PCA

# Create and fit PCA object
pca = PCA(n_components = X.shape[1])
pca.fit_transform(X.detach().numpy())

# Print optimal direction vector
print(pca.components_[0])

[-0.00958687  0.99923202  0.0249962   0.02861215]


In [ ]:
# Select only continuous features
X = torch.tensor(df_food[["Oil", "Density", "Fracture", "Hardness"]].values,
                 dtype=torch.float64)

# ------------------------------
# Step 2: Mean centering
# ------------------------------
mu = torch.mean(X, axis=0)
X_centered = X - mu

# ------------------------------
# Step 3: Initialize direction vector w
# ------------------------------
w = torch.tensor(np.ones(X.shape[1]), dtype=torch.float64, requires_grad=True)
with torch.no_grad():
    w.data = w.data / torch.norm(w)

# ------------------------------
# Step 4: Optimizer
# ------------------------------
optimizer = torch.optim.SGD([w], lr=0.001)

# ------------------------------
# Step 5: Loss function
# ------------------------------
def loss_fn(w, X):
    proj = torch.matmul(X, w)
    var = torch.mean(proj**2)
    loss = -var
    return loss

# ------------------------------
# Step 6: Training loop
# ------------------------------
num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()

    loss = loss_fn(w, X_centered)

    loss.backward()
    optimizer.step()

    # Keep w as a unit vector
    with torch.no_grad():
        w.data = w.data / torch.norm(w)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, loss = {loss.item():.6f}")

# ------------------------------
# Step 7: Final principal component direction
# ------------------------------
print("\nOptimized first principal component (w):")
print(w.detach().numpy())

# ------------------------------
# Step 8: Verification with sklearn PCA
# ------------------------------
pca = PCA(n_components=1)
pca.fit(df_food[["Oil", "Density", "Fracture", "Hardness"]])
print("\nSklearn first principal component direction:")
print(pca.components_[0])

Epoch 0, loss = -4330.057909
Epoch 1000, loss = -15212.920771
Epoch 2000, loss = -15212.920771
Epoch 3000, loss = -15212.920771
Epoch 4000, loss = -15212.920771
Epoch 5000, loss = -15212.920771
Epoch 6000, loss = -15212.920771
Epoch 7000, loss = -15212.920771
Epoch 8000, loss = -15212.920771
Epoch 9000, loss = -15212.920771

Optimized first principal component (w):
[-0.00958687  0.99923202  0.0249962   0.02861215]

Sklearn first principal component direction:
[-0.00958687  0.99923202  0.0249962   0.02861215]


In [ ]:
# Select only continuous features
X = torch.tensor(df_food[["Oil", "Density", "Fracture", "Hardness"]].values,
                 dtype=torch.float64)


# Step 2: Mean centering

mu = torch.mean(X, axis=0)
X_centered = X - mu

# Step 3: Initialize direction vector w

w = torch.tensor(np.ones(X.shape[1]), dtype=torch.float64, requires_grad=True)
with torch.no_grad():
    w.data = w.data / torch.norm(w)


# Step 4: Optimizer

optimizer = torch.optim.SGD([w], lr=0.001)

# Step 5: Loss function

def loss_fn(w, X):
    proj = torch.matmul(X, w)
    var = torch.mean(proj**2)
    loss = -var
    return loss


# Step 6: Training loop

num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()

    loss = loss_fn(w, X_centered)

    loss.backward()
    optimizer.step()

    # Keep w as a unit vector
    with torch.no_grad():
        w.data = w.data / torch.norm(w)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, loss = {loss.item():.6f}")


print("\nOptimized first principal component (w):")
print(w.detach().numpy())


# Step 8: Verification with sklearn PCA

pca = PCA(n_components=1)
pca.fit(df_food[["Oil", "Density", "Fracture", "Hardness"]])
print("\nSklearn first principal component direction:")
print(pca.components_[0])

In [ ]:
import numpy as np

X = np.array([[76, 126, 38], [74, 120, 38], [72, 118, 37.5], [78, 136, 37.0]])
print(X) #matrix of 4 patients and 3 features each

v = np.array([1, 1, 1])
print(v) # 3 element vector

v = v / np.linalg.norm(v)
print(v) #converting v in to unit vector

print(np.dot(X, v)) # How much patien 1 is in agreement with direction v or scalar projection

mu = np.mean(X, axis=0)
print(mu) # average of each feature

print(np.dot(mu, v)) #average of projected patients in the direction of v

print(np.mean(np.dot(X, v))) #average scalar projection

print(np.var(np.dot(X, v))) #variance or how the data points are spread out along the direction of v

print(np.mean(np.square(np.dot(X, v)))-np.mean(np.dot(X, v))) #variance

print(np.mean(np.square(np.dot(X, v)-np.dot(mu, v))))#mean of the squared differences of the projected mean(same as variance)